In [1]:
from langchain_community.llms.llamacpp import LlamaCpp
from transformers import AutoTokenizer
import torch

gguf_path = './functionary-small-v3.2-GGUF/functionary-small-v3.2.Q8_0.gguf'
llm = LlamaCpp(
  model_path=gguf_path,
  n_gpu_layers=-1,
  temperature=0,
  n_ctx=10000,
  verbose=False
)
tokenizer = AutoTokenizer.from_pretrained('./functionary-small-v3.2')


In [2]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from runnables import generate_history_passthrough_function, generate_llm_function, FunctionaryAgentOutputParser

history_passthrough = RunnablePassthrough.assign(history=generate_history_passthrough_function())
llm_runnable = RunnableLambda(generate_llm_function(tokenizer, llm))

agent = history_passthrough | llm_runnable | FunctionaryAgentOutputParser()


In [3]:
from tools import get_current_weather, send_email, search_web

conversation = [
  # {'role': 'user', 'content': 'who is the president of korea now?'}
  {"role": "user", "content": "find the weather in Seoul with search_web and get_current_weather and send email about apple to aa@gmail.com"},
  # {'role': 'assistant', 'content': '[{"name": "get_current_weather", "arguments": {"location": "Seoul, KR", "format": "celsius"}}]'},
  # {'role': 'user', 'content': '38 degree in seoul now'}
]
tools = [get_current_weather.func, send_email.func, search_web.func]

llm_inputs = tokenizer.apply_chat_template(
  conversation=conversation,
  tools=tools,
  add_generation_prompt=True,
  return_dict=True,
  return_tensors="pt",
)
ret = llm.invoke(tokenizer.decode(llm_inputs['input_ids'][0]))
print(ret)

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


search_web
{"text": "Seoul weather"}>>>get_current_weather
{"location": "Seoul", "format": "celsius"}>>>send_email
{"address": "aa@gmail.com", "subject": "Apple", "body": "I love apples!"}


In [4]:
output = agent.invoke({
  'input': 'please tell me current weather in seoul',
  'intermediate_steps': []
})

print(output)


/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


tool='get_current_weather' tool_input={'location': 'Seoul', 'format': 'celsius'} log='no log'


In [5]:
from langchain.agents import AgentExecutor
from tools import get_current_weather, send_email, search_web

tools = [
  get_current_weather,
  send_email,
  search_web
]
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=5, return_intermediate_steps=True)


In [6]:
output = agent_executor.invoke(
  input={
    'input': 'search about average temperature of seoul and compare to current temperature of seoul and send email about the result to aa@gmail.com.'
  }
)
output




> Entering new AgentExecutor chain...


/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


no logthe average day time temperature of seoul is 32 degree celsius

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


no log38 degree

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


no logemail sent successfully

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


Expecting value: line 1 column 1 (char 0)
all
The email has been sent successfully to aa@gmail.com. The email contains the current temperature in Seoul, which is 38 degree Celsius, and compares it with the average daytime temperature of 32 degree Celsius.

> Finished chain.


{'input': 'search about average temperature of seoul and compare to current temperature of seoul and send email about the result to aa@gmail.com.',
 'output': 'all\nThe email has been sent successfully to aa@gmail.com. The email contains the current temperature in Seoul, which is 38 degree Celsius, and compares it with the average daytime temperature of 32 degree Celsius.',
 'intermediate_steps': [(AgentAction(tool='search_web', tool_input={'text': 'average temperature of seoul'}, log='no log'),
   'the average day time temperature of seoul is 32 degree celsius'),
  (AgentAction(tool='get_current_weather', tool_input={'location': 'Seoul', 'format': 'celsius'}, log='no log'),
   '38 degree'),
  (AgentAction(tool='send_email', tool_input={'address': 'aa@gmail.com', 'subject': 'Current Temperature of Seoul', 'body': 'The current temperature in Seoul is 38 degree Celsius. This is higher than the average daytime temperature of 32 degree Celsius.'}, log='no log'),
   'email sent successfully

In [7]:
from json import dumps

dumps('alksdjf')

'"alksdjf"'